<a href="https://colab.research.google.com/github/ArtuomMerkulov/NLP_-_-_-/blob/main/%D0%91%D0%BE%D1%82_%D1%81_NLP_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C%D1%8E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP_моделью для Telegram_бота

## Модель

### Подготовка и обучение модели на архитектуре Seg2Seg

In [2]:
### 1. Загрузка данных
import pandas as pd
import numpy as np
from tensorflow.keras import utils #Используем для to_categoricall
from tensorflow.keras.utils import to_categorical

# Загрузка диалогов из CSV-файла в DataFrame
dataframe = pd.read_csv('dialogue_dataset.csv')
questions = dataframe['question'].values
answers = dataframe['answer'].values

### 2. Предварительная обработка данных
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Токенизация вопросов и ответов
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions + answers)
tokenized_questions = tokenizer.texts_to_sequences(questions)
tokenized_answers = tokenizer.texts_to_sequences(answers)

# Выравнивание последовательностей и создание словарей
maxlen_questions = max([len(x) for x in tokenized_questions])
maxlen_answers = max([len(x) for x in tokenized_answers])

encoder_input_data = pad_sequences(tokenized_questions, maxlen=maxlen_questions, padding='post')
decoder_input_data = pad_sequences(tokenized_answers, maxlen=maxlen_answers, padding='post')

# Подготовка целевых данных для декодера (сдвиг на один шаг вперед)
decoder_target_data = np.zeros_like(decoder_input_data)
decoder_target_data[:, 0:-1] = decoder_input_data[:, 1:]
decoder_target_data = to_categorical(decoder_target_data, num_classes=len(tokenizer.word_index) + 1)

# Словари слова в индекс и индекс в слово
word_to_index = tokenizer.word_index
index_to_word = dict((i, word) for word, i in word_to_index.items())

### 3. Разделение данных на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split

# Деление на обучающую и проверочную выборки
encoder_input_train, encoder_input_test, decoder_input_train, decoder_input_test, decoder_target_train, decoder_target_test = train_test_split(encoder_input_data, decoder_input_data, decoder_target_data, test_size=0.2)

### 4. Построение модели `Seq2Seq`
from keras.layers import Input, LSTM, Dense, Embedding
from keras.models import Model

# Параметры модели
embedding_dim = 128
units = 256

# Энкодер
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(len(word_to_index) + 1, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Декодер
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(len(word_to_index) + 1, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(len(word_to_index) + 1, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Seq2Seq модель
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

### 5. Обучение модели
epochs = 32
batch_size = 64

model.fit(
    [encoder_input_train, decoder_input_train],
    decoder_target_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2
)

### 6. Оценка модели на тестовых данных
model.evaluate([encoder_input_test, decoder_input_test], decoder_target_test)

Epoch 1/32
1/1 [==============================] - 5s 5s/step - loss: 5.5279 - accuracy: 0.0000e+00 - val_loss: 5.4960 - val_accuracy: 0.4706
Epoch 2/32
1/1 [==============================] - 0s 224ms/step - loss: 5.5049 - accuracy: 0.2794 - val_loss: 5.4644 - val_accuracy: 0.4706
Epoch 3/32
1/1 [==============================] - 0s 190ms/step - loss: 5.4845 - accuracy: 0.2843 - val_loss: 5.4254 - val_accuracy: 0.4706
Epoch 4/32
1/1 [==============================] - 0s 177ms/step - loss: 5.4599 - accuracy: 0.2794 - val_loss: 5.3661 - val_accuracy: 0.4706
Epoch 5/32
1/1 [==============================] - 0s 220ms/step - loss: 5.4230 - accuracy: 0.2745 - val_loss: 5.2472 - val_accuracy: 0.4706
Epoch 6/32
1/1 [==============================] - 0s 179ms/step - loss: 5.3477 - accuracy: 0.2745 - val_loss: 4.8732 - val_accuracy: 0.4706
Epoch 7/32
1/1 [==============================] - 0s 217ms/step - loss: 5.0956 - accuracy: 0.2745 - val_loss: 3.5918 - val_accuracy: 0.4706
Epoch 8/32
1/1 [===

[4.103991508483887, 0.38823530077934265]

In [3]:
NLP_model = model # скачивание модели
NLP_model.save('NLP_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [31]:
# Сначала определим энкодер как отдельную модель
encoder_model = Model(encoder_inputs, encoder_states)

# Далее необходимо создать определение для декодера
# Начальное состояние для декодера
decoder_state_input_h = Input(shape=(units,))
decoder_state_input_c = Input(shape=(units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# LSTM декодер с начальными состояниями
decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding,
                                                  initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

# Создание модели декодера
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

# Сохранение моделей
encoder_model.save('encoder_model.h5')
decoder_model.save('decoder_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [33]:
from keras.models import load_model

encoder_model = load_model('/content/encoder_model.h5')
decoder_model = load_model('/content/decoder_model.h5')

In [42]:
def generate_answer(question):
    # Препроцессинг входного вопроса
    question_seq = tokenizer.texts_to_sequences([question])
    question_seq = pad_sequences(question_seq, maxlen=200)

    # Кодирование вопроса
    states_values = encoder_model.predict(question_seq)

    # Генерация пустого ответа с токеном начала последовательности
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer.word_index['<startseq>']

    # Флаг окончания генерации ответа
    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_values)

        # Выбор следующего слова
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = None
        for word, index in tokenizer.word_index.items():
            if sampled_token_index == index:
                if word != '<endseq>':
                    decoded_sentence += ' '+word
                sampled_word = word
                break

        # Обновление целевой последовательности (для следующего шага декодирования)
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Обновление состояния
        states_values = [h, c]

        # Проверка на завершение генерации (по достижении максимальной длины или токена конца)
        if (sampled_word == '<endseq>' or len(decoded_sentence.split()) > 200):
            stop_condition = True

    return decoded_sentence

In [43]:
while True:
    user_question = input("Введите ваш вопрос: ")
    if user_question.lower() == "выход":
        print("До свидания!")
        break
    answer = generate_answer(user_question)
    print("Ответ:", answer)

Введите ваш вопрос: <startseq>Как мне ускорить работу моего компьютера?<endseq>
1/1 [==============================] - 0s 61ms/step


KeyError: '<startseq>'